In [ ]:
# Urease–urea batch reactor batch framework with T-dependent pH-activity
# plus optional linear-in-T pH-activity parameters (extrapolated to 20 °C).

import sys, os, math, re
import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar

# ╔══════════════════════════════════════════════════════════════╗
# ║                       USER CONFIG (BATCH)                    ║
# ╚══════════════════════════════════════════════════════════════╝
CONFIG = {
    "save_path": "/Volumes/01785304894/Simulations/Urease/Enyzme_without_deafctivation_UreaseNOTfixed/batch_runs",

    # Shared reactor/enzyme settings
    "volume_L": 0.2,
    "powder_activity_frac": 1.0,
    "Pt_total_M": 0.0,

    "isothermal": True,      # True: fixed T; False: energy balance
    "adia": True,            # if isothermal=False
    "UA_W_per_K": 100.0,     # if isothermal=False and adia=False
    "T_jacket_C": 40.0,      # energy-balance jacket [°C]
    "rho_kg_per_L": 1.0,
    "Cp_J_per_kgK": 4184.0,
    "deltaH_J_per_mol": -70.5e3,

    # Chemistry initial totals besides S0 (urea)
    "N0": 0.0,
    "C0": 0.0,

    # Time/solver as in your code
    "auto_finish": False,
    "urea_cut_frac": 1e-6,
    "t_final_s": 36000,
    "n_points": 36000,
    "progress_chunks_inner": 60,
    "make_plots": True,

    # NEW: turn on temperature-dependent pH-activity (paper-based, linear in T, extrapolated to 20 °C)
    "use_T_dependent_pH_activity": True,

    # HYPERPARAMETER GRIDS (start/stop/step or explicit "values")
    "grid_initial_pH": {
        "start": 7.3, "stop": 7.3, "step": 0.5,
        # or: "values": [6.0, 7.0, 7.5]
    },
    # urea grid in mM (example: 1, 10, 20, 30, ..., 100)
    "grid_substrate_mM": {
        "start": 1.0, "stop": 1000.0, "step": 10.0,
        # or: "values": [1,10,20,30,40,50,60,70,80,90,100]
    },
    "grid_grams_urease_powder": {
        "start": 0.01, "stop": 1.0, "step": 0.1,   # single value default
        # or: "values": [0.05, 0.1, 0.2]
    },
    # Temperature grid in °C (isothermal setpoint or initial T for energy balance)
    "grid_temperature_C": {
        "start": 40.0, "stop": 40.0, "step": 40.0,  # 20, 30, 40 by default
        # or: "values": [20.0, 25.0, 30.0, 35.0, 40.0]
    },
}

# ╔══════════════════════════════════════════════════════════════╗
# ║                  CONSTANTS & PARAMETERS (ORIGINAL)           ║
# ╚══════════════════════════════════════════════════════════════╝

R   = 8.314  # J/mol/K

# Qin & Cabral (per gram enzyme)
k0_mol_per_s_per_g = 0.267
Ea    = 29.1e3
KM, Ks = 2.56e-3, 6.18

# pH-activity (two-site) — original baseline constants (used if toggle is False)
pKa_es1_const, pKa_es2_const = 9.07, 5.62
Kes1_const,  Kes2_const  = 10**(-pKa_es1_const), 10**(-pKa_es2_const)
alpha_e_const, beta_e_const = 0.373, 0.564

# Product inhibition Kp(pH) (clamped, T-independent per paper up to ~40 °C)
_KP_POINTS = [
    (6.25, 0.1785), (6.50, 0.1194), (7.00, 0.0693), (7.50, 0.0386),
    (8.00, 0.0311), (8.50, 0.0327), (8.75, 0.0298), (9.00, 0.0310),
]
def Kp_of_pH(pH):
    pts = _KP_POINTS
    if pH <= pts[0][0]:
        return pts[0][1]
    if pH >= pts[-1][0]:
        return pts[-1][1]
    for (pa, Ka), (pb, Kb) in zip(pts, pts[1:]):
        if pa <= pH <= pb:
            w = (pH - pa) / (pb - pa)
            return Ka + w * (Kb - Ka)
    return pts[-1][1]

# Optional competitive phosphate inhibition
Ki_phosphate = 0.010  # [M]

# Acid–base constants (held fixed, as in your code)
Kw     = 1e-14
Ka_NH4 = 5.62e-10
Ka1    = 4.45e-7
Ka2    = 4.69e-11
Ka1p   = 7.11e-3
Ka2p   = 6.32e-8
Ka3p   = 4.22e-13

# ╔══════════════════════════════════════════════════════════════╗
# ║         T-DEPENDENT pH-ACTIVITY PARAMS (linear in T °C)      ║
# ╚══════════════════════════════════════════════════════════════╝
# Fits to (25,35,40 °C) with extrapolation allowed (you requested 20 °C).
# pKes1(T) = b1 + m1*T;  pKes2(T) = b2 + m2*T;  alpha(T) = b3 + m3*T;  beta(T) = b4 + m4*T

_m1, _b1 = 0.03428571428571438, 8.217142857142855
_m2, _b2 = -0.031714285714285605, 6.407142857142855
_m3, _b3 = 0.01131428571428572, 0.09085714285714282
_m4, _b4 = 0.013900000000000027, 0.2199999999999993

def get_pH_activity_params(T_K, use_T_dep: bool):
    """Return (Kes1, Kes2, alpha_e, beta_e) for given T.
       If use_T_dep=False, return the original constants."""
    if not use_T_dep:
        return Kes1_const, Kes2_const, alpha_e_const, beta_e_const
    T_C = (T_K - 273.15)
    # Linear relations (valid 25–40 °C; extrapolate at 20 °C per your request)
    pKes1 = _b1 + _m1*T_C
    pKes2 = _b2 + _m2*T_C
    alpha = _b3 + _m3*T_C
    beta  = _b4 + _m4*T_C
    Kes1 = 10**(-pKes1)
    Kes2 = 10**(-pKes2)
    return Kes1, Kes2, float(alpha), float(beta)

# ╔══════════════════════════════════════════════════════════════╗
# ║            ORIGINAL FUNCTIONS (UNCHANGED CORE LOGIC)         ║
# ╚══════════════════════════════════════════════════════════════╝

B_STRONG = 0.0
_SPEC_LAST_LOGH = {"val": 7.0}

def _B_for_target_pH(pH_target, Ntot, Ctot, Ptot):
    H  = 10**(-pH_target)
    OH = Kw / H
    NH4 = Ntot * (H / (H + Ka_NH4))
    denom_c = H*H + Ka1*H + Ka1*Ka2
    HCO3 = Ctot * (Ka1 * H / denom_c)
    CO3  = Ctot * (Ka1 * Ka2 / denom_c)
    Dp   = H**3 + Ka1p*H**2 + Ka1p*Ka2p*H + Ka1p*Ka2p*Ka3p
    H2PO4 = Ptot * (Ka1p * H**2 / Dp)
    HPO4  = Ptot * (Ka1p * Ka2p * H / Dp)
    PO4   = Ptot * (Ka1p * Ka2p * Ka3p / Dp)
    B = (OH + HCO3 + 2*CO3 + H2PO4 + 2*HPO4 + 3*PO4) - (H + NH4)
    return B

def _charge_balance(logH, Ntot, Ctot, Ptot):
    H  = 10**(-logH)
    OH = Kw / H
    NH4 = Ntot * (H / (H + Ka_NH4))
    denom_c = H*H + Ka1*H + Ka1*Ka2
    HCO3 = Ctot * (Ka1 * H / denom_c)
    CO3  = Ctot * (Ka1 * Ka2 / denom_c)
    Dp   = H**3 + Ka1p*H**2 + Ka1p*Ka2p*H + Ka1p*Ka2p*Ka3p
    H2PO4 = Ptot * (Ka1p * H**2 / Dp)
    HPO4  = Ptot * (Ka1p * Ka2p * H / Dp)
    PO4   = Ptot * (Ka1p * Ka2p * Ka3p / Dp)
    return (H + NH4) - (OH + HCO3 + 2*CO3 + H2PO4 + 2*HPO4 + 3*PO4) + B_STRONG

def compute_speciation(Ntot, Ctot, Ptot):
    Ntot = max(Ntot, 0.0); Ctot = max(Ctot, 0.0); Ptot = max(Ptot, 0.0)
    lo = max(1.0, _SPEC_LAST_LOGH["val"] - 3.0)
    hi = min(13.0, _SPEC_LAST_LOGH["val"] + 3.0)
    f_lo = _charge_balance(lo, Ntot, Ctot, Ptot)
    f_hi = _charge_balance(hi, Ntot, Ctot, Ptot)
    if f_lo * f_hi > 0:
        lo, hi = 1.0, 13.0
    sol = root_scalar(_charge_balance, bracket=[lo, hi], method='brentq',
                      args=(Ntot, Ctot, Ptot))
    logH = sol.root
    _SPEC_LAST_LOGH["val"] = logH
    H  = 10**(-logH)
    OH = Kw / H
    NH4 = Ntot * (H / (H + Ka_NH4))
    NH3 = Ntot - NH4
    denom_c = H*H + Ka1*H + Ka1*Ka2
    CO2  = Ctot * (H*H / denom_c)
    HCO3 = Ctot * (Ka1 * H / denom_c)
    CO3  = Ctot * (Ka1 * Ka2 / denom_c)
    Dp   = H**3 + Ka1p*H**2 + Ka1p*Ka2p*H + Ka1p*Ka2p*Ka3p
    H2PO4 = Ptot * (Ka1p * H**2 / Dp)
    HPO4  = Ptot * (Ka1p * Ka2p * H / Dp)
    PO4   = Ptot * (Ka1p * Ka2p * Ka3p / Dp)
    return {
        'pH': -math.log10(H),
        'H': H, 'OH': OH,
        'NH3': NH3, 'NH4': NH4,
        'CO2': CO2, 'HCO3': HCO3, 'CO3': CO3,
        'H2PO4': H2PO4, 'HPO4': HPO4, 'PO4': PO4
    }

def rate_per_g(S, Ntot, pH, T, P_inhib, use_T_dep_params):
    S = max(S, 0.0)
    H = 10**(-pH)
    # Arrhenius
    kT = k0_mol_per_s_per_g * math.exp(-Ea / (R * T))
    # pH-activity (now optionally T-dependent parameters)
    Kes1, Kes2, alpha_e, beta_e = get_pH_activity_params(T, use_T_dep_params)
    pH_factor = 1.0 / (1.0 + (Kes1/H)**alpha_e + (H/Kes2)**beta_e)
    # competitive phosphate
    alpha_comp = 1.0 + (P_inhib / Ki_phosphate if Ki_phosphate > 0.0 else 0.0)
    denom = max(KM * alpha_comp + S + (S*S)/Ks, 1e-12)
    v_sub = kT * pH_factor * (S / denom)
    # noncompetitive product inhibition (Kp is pH-only in this model)
    v = v_sub / (1.0 + max(Ntot, 0.0) / Kp_of_pH(pH))
    return v

def rhs_isothermal(t, y, fixed_temperature_K, E_loading_g_per_L, Pt_total_M, use_T_dep_params):
    S, Ntot, Ctot = y
    sp  = compute_speciation(Ntot, Ctot, Pt_total_M)
    pH  = sp['pH']
    Pin = sp['H2PO4'] + sp['HPO4']
    per_g = rate_per_g(S, Ntot, pH, fixed_temperature_K, Pin, use_T_dep_params)
    r_NH3 = per_g * E_loading_g_per_L
    dS_dt = -0.5 * r_NH3
    dN_dt =       r_NH3
    dC_dt =  0.5 * r_NH3
    return [dS_dt, dN_dt, dC_dt]

def rhs_energy(t, y, E_loading_g_per_L, Pt_total_M, adia, UA_W_per_K, volume_L,
               T_jacket_K, rho_kg_per_L, Cp_J_per_kgK, deltaH_J_per_mol, use_T_dep_params):
    S, Ntot, Ctot, T = y
    sp  = compute_speciation(Ntot, Ctot, Pt_total_M)
    pH  = sp['pH']
    Pin = sp['H2PO4'] + sp['HPO4']
    per_g = rate_per_g(S, Ntot, pH, T, Pin, use_T_dep_params)
    r_NH3 = per_g * E_loading_g_per_L
    dS_dt = -0.5 * r_NH3
    dN_dt =       r_NH3
    dC_dt =  0.5 * r_NH3
    r_urea = -dS_dt
    q_rxn_per_L = (-deltaH_J_per_mol) * r_urea
    if adia:
        dT_dt = q_rxn_per_L / (rho_kg_per_L * Cp_J_per_kgK)
    else:
        UA_per_L = UA_W_per_K / volume_L
        dT_dt = (q_rxn_per_L - UA_per_L * (T - T_jacket_K)) / (rho_kg_per_L * Cp_J_per_kgK)
    return [dS_dt, dN_dt, dC_dt, dT_dt]

def event_urea_depleted(t, y, S0, urea_cut_frac):
    S = y[0]
    return S - urea_cut_frac * S0
event_urea_depleted.terminal  = True
event_urea_depleted.direction = -1

def _print_bar(frac, width=40, prefix=""):
    frac = max(0.0, min(1.0, frac))
    done = int(width * frac)
    bar  = '█' * done + '░' * (width - done)
    msg  = f"\r{prefix}[{bar}] {100*frac:6.2f}%"
    sys.stdout.write(msg); sys.stdout.flush()

def integrate_with_progress(rhs, y0, times, chunks=50, show_progress=False):
    T_out = []
    Y_out = []
    edges = np.linspace(times[0], times[-1], chunks + 1)
    y_curr = np.array(y0, dtype=float)
    for k in range(chunks):
        t_a, t_b = edges[k], edges[k+1]
        mask = (times >= t_a) & (times <= t_b)
        t_eval_chunk = times[mask]
        if t_eval_chunk.size == 0:
            continue
        sol = solve_ivp(rhs, [t_a, t_b], y_curr, method='BDF',
                        t_eval=t_eval_chunk, max_step=60.0,
                        rtol=1e-6, atol=1e-12)
        if not sol.success:
            raise RuntimeError(f"Solve failed in chunk {k}: {sol.message}")
        if len(T_out) == 0:
            T_out = list(sol.t)
            Y_out = [arr.copy() for arr in sol.y]
        else:
            T_out.extend(list(sol.t[1:]))
            for i in range(len(sol.y)):
                Y_out[i] = np.concatenate((Y_out[i], sol.y[i][1:]))
        y_curr = sol.y[:, -1]
        if show_progress:
            _print_bar((t_b - times[0]) / (times[-1] - times[0] + 1e-12), prefix=" run ")
    if show_progress:
        sys.stdout.write("\n")
    return np.array(T_out), np.vstack(Y_out)

# ╔══════════════════════════════════════════════════════════════╗
# ║             SINGLE-RUN WRAPPER (original flow)               ║
# ╚══════════════════════════════════════════════════════════════╝

def run_single_simulation(
    save_dir,
    volume_L,
    grams_urease_powder,
    powder_activity_frac,
    initial_pH,
    isothermal,
    fixed_temperature_K,
    adia,
    UA_W_per_K,
    T_jacket_K,
    rho_kg_per_L,
    Cp_J_per_kgK,
    deltaH_J_per_mol,
    Pt_total_M,
    S0, N0, C0,
    auto_finish, urea_cut_frac, t_final_s,
    n_points,
    progress_chunks_inner,
    make_plots,
    use_T_dep_params
):
    os.makedirs(save_dir, exist_ok=True)

    # Enzyme loading
    E_loading_g_per_L = grams_urease_powder * powder_activity_frac / volume_L

    # Initialize strong-ion background from initial pH
    global B_STRONG, _SPEC_LAST_LOGH
    if initial_pH is not None:
        B_STRONG = _B_for_target_pH(initial_pH, N0, C0, Pt_total_M)
        _SPEC_LAST_LOGH["val"] = initial_pH
        print(f"[init] pH0={initial_pH:.2f} ⇒ (Na+ - Cl-) = {B_STRONG:.4f} M")
    else:
        B_STRONG = 0.0

    # RHS and initial state
    if isothermal:
        y0  = [S0, N0, C0]
        def rhs(t, y):
            return rhs_isothermal(t, y, fixed_temperature_K, E_loading_g_per_L, Pt_total_M, use_T_dep_params)
    else:
        y0  = [S0, N0, C0, fixed_temperature_K]
        def rhs(t, y):
            return rhs_energy(t, y, E_loading_g_per_L, Pt_total_M, adia, UA_W_per_K, volume_L,
                              T_jacket_K, rho_kg_per_L, Cp_J_per_kgK, deltaH_J_per_mol, use_T_dep_params)

    # Horizon (auto_finish vs fixed)
    if auto_finish:
        sol_probe = solve_ivp(lambda t,y: rhs(t,y), [0, 1e7], y0, method='BDF',
                              events=lambda t,y: event_urea_depleted(t,y,S0,urea_cut_frac),
                              max_step=60.0, rtol=1e-6, atol=1e-12)
        if len(sol_probe.t_events) > 0 and len(sol_probe.t_events[0]) > 0:
            t_target = float(sol_probe.t_events[0][0])
        else:
            t_target = float(sol_probe.t[-1])
    else:
        t_target = float(t_final_s)

    # Integrate on uniform grid (no inner bar)
    times = np.linspace(0.0, t_target, n_points)
    T_out, Y_out = integrate_with_progress(rhs, y0, times,
                                           chunks=progress_chunks_inner,
                                           show_progress=False)

    # Build dataframe (identical columns)
    records = []
    if isothermal:
        for t, S, Ntot, Ctot in zip(T_out, *Y_out):
            sp = compute_speciation(Ntot, Ctot, Pt_total_M)
            records.append({
                'time [s]': t,
                'urea [M]': max(S, 0.0),
                'total ammonia [M]': max(Ntot, 0.0),
                'total inorganic carbon [M]': max(Ctot, 0.0),
                'pH': sp['pH'],
                'NH3 [M]': sp['NH3'],
                'NH4+ [M]': sp['NH4'],
                'CO2(aq) [M]': sp['CO2'],
                'HCO3- [M]': sp['HCO3'],
                'CO3(2-) [M]': sp['CO3'],
                'H2PO4- [M]': sp['H2PO4'],
                'HPO4(2-) [M]': sp['HPO4'],
                'PO4(3-) [M]': sp['PO4'],
                'T [K]': fixed_temperature_K,
                'T [°C]': fixed_temperature_K - 273.15
            })
    else:
        for t, S, Ntot, Ctot, T in zip(T_out, *Y_out):
            sp = compute_speciation(Ntot, Ctot, Pt_total_M)
            records.append({
                'time [s]': t,
                'urea [M]': max(S, 0.0),
                'total ammonia [M]': max(Ntot, 0.0),
                'total inorganic carbon [M]': max(Ctot, 0.0),
                'pH': sp['pH'],
                'NH3 [M]': sp['NH3'],
                'NH4+ [M]': sp['NH4'],
                'CO2(aq) [M]': sp['CO2'],
                'HCO3- [M]': sp['HCO3'],
                'CO3(2-) [M]': sp['CO3'],
                'H2PO4- [M]': sp['H2PO4'],
                'HPO4(2-) [M]': sp['HPO4'],
                'PO4(3-) [M]': sp['PO4'],
                'T [K]': T,
                'T [°C]': T - 273.15
            })

    df = pd.DataFrame(records)
    csv_main = os.path.join(save_dir, "simulation_results.csv")
    df.to_csv(csv_main, index=False)

    # pH curve and species CSVs
    pH_csv = os.path.join(save_dir, "pH_curve.csv")
    species_csv = os.path.join(save_dir, "species_profiles.csv")
    df[['time [s]', 'pH']].to_csv(pH_csv, index=False)
    species_cols = ['urea [M]', 'NH3 [M]', 'NH4+ [M]', 'CO2(aq) [M]', 'HCO3- [M]', 'CO3(2-) [M]']
    species_cols = [c for c in species_cols if c in df.columns]
    df[['time [s]'] + species_cols].to_csv(species_csv, index=False)

    # Plots (unchanged, saved inside save_dir)
    if make_plots:
        def plain_axis(ax): ax.ticklabel_format(useOffset=False, style='plain', axis='y')
        title_pH = f" (pH₀={initial_pH:.2f})" if initial_pH is not None else ""
        # pH vs time
        plt.figure(); ax = plt.gca()
        plt.plot(df['time [s]'], df['pH'])
        plain_axis(ax)
        plt.xlabel('Time [s]'); plt.ylabel('pH')
        ttl = 'pH evolution' + title_pH + (' @ 40 °C (isothermal)' if isothermal else '')
        plt.title(ttl)
        plt.savefig(os.path.join(save_dir, 'pH_vs_time.png'), bbox_inches='tight'); plt.close()

        # species vs time
        plt.figure(); ax = plt.gca()
        for col, lbl in [
            ('urea [M]', 'urea'),
            ('NH3 [M]',  'NH$_3$'),
            ('NH4+ [M]', 'NH$_4^+$'),
            ('CO2(aq) [M]', 'CO$_2$(aq)'),
            ('HCO3- [M]',   'HCO$_3^-$'),
            ('CO3(2-) [M]', 'CO$_3^{2-}$'),
        ]:
            if col in df.columns:
                plt.plot(df['time [s]'], df[col], label=lbl)
        plain_axis(ax)
        plt.xlabel('Time [s]'); plt.ylabel('Concentration [M]')
        plt.legend(); plt.title('Species profiles' + title_pH)
        plt.savefig(os.path.join(save_dir, 'species_vs_time.png'), bbox_inches='tight'); plt.close()

        # phosphate speciation (only if present)
        if Pt_total_M > 0:
            plt.figure(); ax = plt.gca()
            for col, lbl in [
                ('H2PO4- [M]', 'H$_2$PO$_4^-$'),
                ('HPO4(2-) [M]', 'HPO$_4^{2-}$'),
                ('PO4(3-) [M]', 'PO$_4^{3-}$'),
            ]:
                plt.plot(df['time [s]'], df[col], label=lbl)
            plain_axis(ax); plt.xlabel('Time [s]'); plt.ylabel('Concentration [M]')
            plt.legend(); plt.title('Phosphate speciation' + title_pH)
            plt.savefig(os.path.join(save_dir, 'phosphate_vs_time.png'), bbox_inches='tight'); plt.close()

        # temperature vs time (only if non-isothermal)
        if not isothermal:
            plt.figure(); ax = plt.gca()
            plt.plot(df['time [s]'], df['T [°C]'])
            plain_axis(ax); plt.xlabel('Time [s]'); plt.ylabel('Temperature [°C]')
            plt.title('Temperature evolution (energy balance)' + title_pH)
            plt.savefig(os.path.join(save_dir, 'T_vs_time.png'), bbox_inches='tight'); plt.close()

    if auto_finish:
        print(f"Finished at t = {times[-1]:.1f} s when S < {urea_cut_frac*100:.4g}% of S0.")
    else:
        print("Finished fixed-time run.")
    print(f"Data → {csv_main}; figures saved in folder.")
    return df

# ╔══════════════════════════════════════════════════════════════╗
# ║                 BATCH GRID + GLOBAL PROGRESS                 ║
# ╚══════════════════════════════════════════════════════════════╝

def _grid_from_dict(d, default_single=None, *, is_mM=False):
    if d is None:
        return [default_single]
    if "values" in d and d["values"] is not None:
        return list(d["values"])
    start, stop, step = d.get("start"), d.get("stop"), d.get("step")
    if start is None or stop is None or step is None:
        return [default_single]
    if is_mM:
        vals = [start] + list(np.arange(step, stop + 1e-12, step))
        return sorted(set([float(v) for v in vals if v <= stop + 1e-12]))
    n = int(np.floor((stop - start)/step + 1e-9)) + 1
    vals = [start + i*step for i in range(max(n,1))]
    if vals[-1] < stop - 1e-12:
        vals.append(stop)
    return [float(v) for v in vals]

def _slugify(s):
    s = re.sub(r'[^\w\-.]+', '_', s)
    return s.strip('_')

def main_batch(cfg: dict):
    root = cfg["save_path"]; os.makedirs(root, exist_ok=True)

    grid_pH = _grid_from_dict(cfg.get("grid_initial_pH"), default_single=None)
    grid_mM = _grid_from_dict(cfg.get("grid_substrate_mM"), default_single=20.0, is_mM=True)
    grid_Eg = _grid_from_dict(cfg.get("grid_grams_urease_powder"), default_single=0.1)
    grid_Tc = _grid_from_dict(cfg.get("grid_temperature_C"), default_single=40.0)

    combos = []
    for pH0 in grid_pH:
        for mM in grid_mM:
            for grams in grid_Eg:
                for Tc in grid_Tc:
                    combos.append((pH0, mM, grams, Tc))
    total = len(combos)
    if total == 0:
        print("No combinations to run. Check your grid settings."); return

    volume_L = cfg["volume_L"]
    powder_activity_frac = cfg["powder_activity_frac"]
    Pt_total_M = cfg["Pt_total_M"]
    isothermal = cfg["isothermal"]
    adia = cfg["adia"]
    UA_W_per_K = cfg["UA_W_per_K"]
    T_jacket_K = cfg["T_jacket_C"] + 273.15
    rho_kg_per_L = cfg["rho_kg_per_L"]
    Cp_J_per_kgK = cfg["Cp_J_per_kgK"]
    deltaH_J_per_mol = cfg["deltaH_J_per_mol"]

    N0 = cfg["N0"]; C0 = cfg["C0"]

    auto_finish = cfg["auto_finish"]
    urea_cut_frac = cfg["urea_cut_frac"]
    t_final_s = cfg["t_final_s"]
    n_points = cfg["n_points"]
    progress_chunks_inner = cfg["progress_chunks_inner"]
    make_plots = cfg["make_plots"]
    use_T_dep_params = bool(cfg.get("use_T_dependent_pH_activity", False))

    print(f"Total simulations: {total}  |  T-dependent pH-activity: {use_T_dep_params}")
    done = 0
    _print_bar(0.0, prefix=" batch ")

    for idx, (pH0, mM, grams, Tc) in enumerate(combos, start=1):
        S0 = float(mM) * 1e-3   # mM → M
        fixed_temperature_K = float(Tc) + 273.15

        slug = _slugify(f"pH{('None' if pH0 is None else f'{pH0:.2f}')}"
                        f"_S{mM:.3g}mM_E{grams:.3g}g_T{Tc:.3g}C")
        save_dir = os.path.join(root, f"run_{idx:04d}_{slug}")
        os.makedirs(save_dir, exist_ok=True)

        with open(os.path.join(save_dir, "MANIFEST.txt"), "w") as f:
            f.write(
                f"initial_pH: {pH0}\n"
                f"urea_S0_mM: {mM}\n"
                f"grams_urease_powder: {grams}\n"
                f"temperature_C: {Tc}\n"
                f"isothermal: {isothermal}\n"
                f"use_T_dependent_pH_activity: {use_T_dep_params}\n"
                f"volume_L: {volume_L}\n"
                f"powder_activity_frac: {powder_activity_frac}\n"
                f"Pt_total_M: {Pt_total_M}\n"
                f"auto_finish: {auto_finish}\n"
                f"urea_cut_frac: {urea_cut_frac}\n"
                f"t_final_s: {t_final_s}\n"
                f"n_points: {n_points}\n"
            )

        run_single_simulation(
            save_dir=save_dir,
            volume_L=volume_L,
            grams_urease_powder=grams,
            powder_activity_frac=powder_activity_frac,
            initial_pH=pH0,
            isothermal=isothermal,
            fixed_temperature_K=fixed_temperature_K,
            adia=adia,
            UA_W_per_K=UA_W_per_K,
            T_jacket_K=T_jacket_K,
            rho_kg_per_L=rho_kg_per_L,
            Cp_J_per_kgK=Cp_J_per_kgK,
            deltaH_J_per_mol=deltaH_J_per_mol,
            Pt_total_M=Pt_total_M,
            S0=S0, N0=N0, C0=C0,
            auto_finish=auto_finish,
            urea_cut_frac=urea_cut_frac,
            t_final_s=t_final_s,
            n_points=n_points,
            progress_chunks_inner=progress_chunks_inner,
            make_plots=make_plots,
            use_T_dep_params=use_T_dep_params
        )

        done += 1
        _print_bar(done/total, prefix=" batch ")

    sys.stdout.write("\n")
    print("All simulations complete.")

if __name__ == "__main__":
    main_batch(CONFIG)


Total simulations: 1111  |  T-dependent pH-activity: True
 batch [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   0.00%[init] pH0=7.30 ⇒ (Na+ - Cl-) = 0.0000 M
Finished fixed-time run.
Data → /Volumes/01785304894/Simulations/Urease/Enyzme_without_deafctivation_UreaseNOTfixed/batch_runs/run_0001_pH7.30_S1mM_E0.01g_T40C/simulation_results.csv; figures saved in folder.
 batch [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   0.09%[init] pH0=7.30 ⇒ (Na+ - Cl-) = 0.0000 M
Finished fixed-time run.
Data → /Volumes/01785304894/Simulations/Urease/Enyzme_without_deafctivation_UreaseNOTfixed/batch_runs/run_0002_pH7.30_S1mM_E0.11g_T40C/simulation_results.csv; figures saved in folder.
 batch [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   0.18%[init] pH0=7.30 ⇒ (Na+ - Cl-) = 0.0000 M
Finished fixed-time run.
Data → /Volumes/01785304894/Simulations/Urease/Enyzme_without_deafctivation_UreaseNOTfixed/batch_runs/run_0003_pH7.30_S1mM_E0.21g_T40C/simulation_results.csv; figures saved in folder.
 batch [░░░░░░░░░░░░░